In [0]:
import tensorflow as tf
from tensorflow.layers import dense, conv1d,  conv2d, dropout, max_pooling2d, max_pooling1d, batch_normalization
from tensorflow.contrib.layers import flatten
from tensorflow.nn import relu, relu6
import numpy as np
import time
import math


In [0]:
## Implementation of VDCNN Architecture
# Very Deep Convolution Neural Network for text classification
# use word embedding instead of char embedding
# link of paper : 
def Conv_Block(inputs, filters, kernel_size, name, stride=1, optional_shortcut = True):
  """
    Convolution block, each convolution is followed by batch norm and relu function
    @optional_shortcut : parameters which activate or not the residual
  """
  
  
  
  
  conv_1 = conv1d(inputs=inputs, filters=filters, kernel_size=kernel_size, strides=stride, padding="same", name= name + "_conv_1")
  batch_norm_1 = batch_normalization(conv_1, name=name + "_batchnorm_1")
  relu_1 = relu(batch_norm_1, name=name + "relu_1")
  
  conv_2 = conv1d(inputs=relu_1, filters=filters, kernel_size=kernel_size, strides=stride, padding="same", name= name + "_conv_2")
  batch_norm_2 = batch_normalization(conv_2, name=name + "_batchnorm_2")
  relu_2 = relu(batch_norm_2, name=name + "relu_2")
  
  if optional_shortcut:
    shorcut = conv1d(inputs=inputs, filters=filters, kernel_size=1, strides=stride, padding="same", name= name + "_shorcut")
    batch_norm_shorcut = batch_normalization(shorcut, name=name + "_shorcut_batchnorm_2")
    relu_shortcut = relu(batch_norm_shorcut, name=name + "_shorcut_relu_2")
    return relu_shortcut + relu_2
  
  else:
    return relu_2


  
  
def VDCNN(inputs, vocabulary_size, embedding_size=16, kmax=8, size_sentences=100, classes=2):
  """
    @inputs : array of sentences (each word is represented by a number)
    @vocabulary_size : size of the vocabulary
    @embedding_size : a small size is adviced when we use char embedding. If we use word embedding, bigger dimension are required
  """
  # create embedding
  #init_embeddings = tf.random_uniform(, -1.0, 1.0)
  word_embeddings = tf.get_variable("word_embeddings", [vocabulary_size, embedding_size])
  embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, inputs, name="lookupTable")
  #n = size_sentences
  # conv layers
  
  x = conv1d(inputs=embedded_word_ids, filters=64, kernel_size=3, strides=1, padding="same", name= "conv_0")
  with tf.name_scope("ConvBlock"):
    nblocks = 3
    for i in range(0, nblocks):
      x = Conv_Block(x, filters=64*(2**i), kernel_size=3, name="ConvBlock1_"+str(i))
      x = Conv_Block(x, filters=64*(2**i), kernel_size=3, name="ConvBlock2_"+str(i))
      x = max_pooling1d(x, pool_size=3, strides=2, name = "Maxpool_" + str(i))
     # n = n/2
    i = i + 1

    x = Conv_Block(x, filters=64*(2**i), kernel_size=3, name="ConvBlock1_"+str(i))
    x = Conv_Block(x, filters=64*(2**i), kernel_size=3, name="ConvBlock2_"+str(i))

    #n = int(n//kmax)
  
  # === K Max Pooling === #
  with tf.name_scope("k-max-pooling"):
            x = tf.transpose(x, [0, 2, 1])
            x = tf.nn.top_k(x, k=kmax, sorted=False).values

  
  with tf.name_scope("Full-Connected-Layers"):
    
    x = flatten(x)

    # === FC Layers === #
    x = dropout(x, 0.5)
    x = dense(x, 2048, name="dense_1", activation="relu")
    x = dropout(x, 0.5)
    x = dense(x, 2048, name="dense_2", activation="relu")
    x = dropout(x, 0.5)
    x = dense(x, classes, name="dense_output", activation=None)

  return x

def VDCNN_light(inputs, vocabulary_size, embedding_size=16, kmax=8, size_sentences=100, classes=2):
  """
    @inputs : array of sentences (each word is represented by a number)
    @vocabulary_size : size of the vocabulary
    @embedding_size : a small size is adviced when we use char embedding. If we use word embedding, bigger dimension are required
  """
  # create embedding
  #init_embeddings = tf.random_uniform(, -1.0, 1.0)
  word_embeddings = tf.get_variable("word_embeddings", [vocabulary_size, embedding_size])
  embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, inputs, name="lookupTable")

  # conv layers
  
  x = conv1d(inputs=embedded_word_ids, filters=64, kernel_size=3, strides=1, padding="same", name= "conv_0")
  
  with tf.name_scope("ConvBlock"):
    nblocks = 3
    for i in range(0, nblocks):
      x = Conv_Block(x, filters=64*(2**i), kernel_size=3, name="ConvBlock1_"+str(i))
      x = max_pooling1d(x, pool_size=3, strides=2, name = "Maxpool_" + str(i))
    i = i + 1

    x = Conv_Block(x, filters=64*(2**i), kernel_size=3, name="ConvBlock1_"+str(i))
  
  # === K Max Pooling === #
  with tf.name_scope("k-max-pooling"):
            x = tf.transpose(x, [0, 2, 1])
            x = tf.nn.top_k(x, k=kmax, sorted=False).values

  
  with tf.name_scope("Full-Connected-Layers"):
    
    x = flatten(x)

    # === FC Layers === #
    x = dropout(x, 0.5)
    x = dense(x, 512, name="dense_1", activation="relu")
    #x = dropout(x, 0.5)
    #x = dense(x, 512, name="dense_2", activation="relu")
    x = dropout(x, 0.5)
    x = dense(x, classes, name="dense_output", activation=None)

  return x

In [4]:
from tensorflow import keras
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
train_labels= train_labels.reshape(-1,1)
test_labels = test_labels.reshape(-1,1)
max_=0
min_=10000
for s in train_data:
  max_ = max(max_, len(s))
  min_ = min(min_, len(s))
for s in test_data:
  max_ = max(max_, len(s))
  min_ = min(min_, len(s))
print("Size min/max sentences : {0}/{1}".format(min_, max_))
print("train : {0} {1}".format(len(train_data), train_labels.shape))
print("test : {0} {1}".format(len(test_data), test_labels.shape))

17465344/17464789 [==============================] - 0s 0us/step
Size min/max sentences : 7/2494
train : 25000 (25000, 1)
test : 25000 (25000, 1)


In [5]:
## preprocessing on data, neural network need same dimension inputs for batch
def decode_review(text, dic):
    return ' '.join([dic[i] for i in text])

# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decode_review(train_data[0], reverse_word_index)


1646592/1641221 [==============================] - 0s 0us/step


"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [6]:
print(train_labels[0,:])
print(type(train_data[0]))
print(train_data.shape, train_labels.shape)
print(train_data[0])

[1]
<class 'list'>
(25000,) (25000, 1)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [7]:
# preprocessing, add padding
max_=128 #size max of a sentence
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=max_)
train_data = train_data.astype(np.int32)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=max_)
test_data = test_data.astype(np.int32)

from keras.utils import to_categorical


print(train_labels.shape, test_labels.shape)
print(train_labels[0], test_labels[0])
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_labels.shape, test_labels.shape)
print(train_labels[0], test_labels[0])

(25000, 1) (25000, 1)
[1] [0]
(25000, 2) (25000, 2)
[0. 1.] [1. 0.]


Using TensorFlow backend.


In [15]:
# tf graph
classes = 2
tf.reset_default_graph()
# 0 negatif, 1 positif
x =  tf.placeholder(tf.int32, [None, max_], name="x")
y = tf.placeholder(tf.int32, [None, classes], name="y")
model = VDCNN(inputs=x, vocabulary_size=10000, embedding_size=300, kmax=8, size_sentences= max_, classes=classes)

# ====== Loop Training ===== #
batch_size = 128
batch_size_test = 256
total_training = len(train_data)
total_test = len(test_data)
nb_epochs = 10

preds = tf.nn.softmax(model)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
accuracy = tf.reduce_mean( tf.cast(tf.equal(tf.argmax(preds, 1, output_type=tf.int32), tf.argmax(y, 1, output_type=tf.int32)), tf.float32))
train_step = tf.contrib.opt.NadamOptimizer(0.0001).minimize(loss)
init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  numberIteration = total_training//batch_size
  numberIteration_test = int(np.ceil(total_test/batch_size_test))
  print("Number of Iteration per epoch : {}".format(numberIteration))
  print("Number of Iteration per epoch test : {}".format(numberIteration_test))
  for e in range(0, nb_epochs):
    
    # acc and loss for each batch
    acc_ = np.zeros((numberIteration,))
    loss_ = np.zeros((numberIteration,))
    
    begin = time.time()
    for i in range(0, numberIteration):

      
      choice = np.random.randint(0, total_training, batch_size, dtype=np.int32)
      
      train_batch = {x : train_data[choice,:] , y : train_labels[choice,:]}
      
      _, accuracy_iteration, loss_iteration, preds_iteration = sess.run([train_step, accuracy, loss, preds], feed_dict=train_batch)
      
      acc_[i] = accuracy_iteration
      loss_[i] = loss_iteration
      #print("# === Training Iteration n°{0} === # \n loss CE : {1} -- accuracy : {2}". format(i, np.mean(loss_[:i+1]), np.mean(acc_[:i+1])))
      #print(preds_iteration)
      #cal = tf.reduce_mean(-tf.reduce_sum(train_batch[y] * tf.log(preds_iteration), 1))
      #print(cal.eval())
      #cal2= np.equal(np.argmax(preds_iteration, axis=1), np.argmax(train_batch[y], axis=1))
      #print(np.sum(cal2), np.sum(cal2)/batch_size)
      #print(loss_iteration, accuracy_iteration)
    print("# === Training epoch n°{0}  {1} sec === # \n loss CE : {2} -- accuracy : {3}". format(e, time.time()-begin, np.mean(loss_), np.mean(acc_)))
    
    # == Testing step == #
    accuracy_test = np.zeros((numberIteration_test,))
    loss_test = np.zeros((numberIteration_test,))
    begin = time.time()
    
    for i in range(0, numberIteration_test):
      if i == numberIteration_test -1:
        test_batch ={x : test_data[i*batch_size_test:, :], y : test_labels[i*batch_size_test:, :] }
        accuracy_iteration_test, loss_iteration_test = sess.run([accuracy, loss], feed_dict=test_batch)
        
        accuracy_test[i] = accuracy_iteration_test * (total_test-i*batch_size_test)
        loss_test[i] = loss_iteration_test * (total_test-i*batch_size_test)
        
      else:
        test_batch ={x : test_data[i*batch_size_test:(i+1)*batch_size_test, :], y : test_labels[i*batch_size_test:(i+1)*batch_size_test, :] }
        accuracy_iteration_test, loss_iteration_test = sess.run([accuracy, loss], feed_dict=test_batch)
        
        accuracy_test[i] = accuracy_iteration_test * batch_size_test
        loss_test[i] = loss_iteration_test * batch_size_test
      #print("# === Testing Iteration n°{0} === # \n loss CE : {1} -- accuracy : {2}". format(i, np.sum(loss_test[:i+1])/((i+1)*batch_size_test), np.sum(accuracy_test[:i+1])/((i+1)*batch_size_test)))

    print("# === Testing epoch n°{0}  {1} sec === # \n loss CE : {2} -- accuracy : {3}". format(e, time.time()-begin, np.sum(loss_test)/total_test, np.sum(accuracy_test)/total_test))
    #print(loss_test)
  
#278



Number of Iteration per epoch : 195
Number of Iteration per epoch test : 98
# === Training epoch n°0  36.298972845077515 sec === # 
 loss CE : 0.6705681672463051 -- accuracy : 0.5405048076923077
# === Testing epoch n°0  18.152676820755005 sec === # 
 loss CE : 0.5358535016345978 -- accuracy : 0.728800000076294
# === Training epoch n°1  35.23712611198425 sec === # 
 loss CE : 0.30142282572312235 -- accuracy : 0.875
# === Testing epoch n°1  17.86713171005249 sec === # 
 loss CE : 0.3642820947170258 -- accuracy : 0.847560000076294
# === Training epoch n°2  35.085737466812134 sec === # 
 loss CE : 0.19372028811619832 -- accuracy : 0.9290064102564103
# === Testing epoch n°2  17.871349573135376 sec === # 
 loss CE : 0.3937003105306625 -- accuracy : 0.8577200000762939
# === Training epoch n°3  35.1430127620697 sec === # 
 loss CE : 0.12421411780210642 -- accuracy : 0.9587339743589743
# === Testing epoch n°3  17.851994037628174 sec === # 
 loss CE : 0.4242630764722824 -- accuracy : 0.853920000

In [0]:
#!kill -9 -1